In [1]:
import autokeras
import pandas
import numpy
import os
from PIL import Image

In [2]:
def load_dataset(dirname, label):
    X = []
    Y = []
    filenames = os.listdir(dirname)
    for filename in filenames:
        img = Image.open("{}/{}".format(dirname, filename))
        arr = numpy.array(img)
        X.append(arr)
        Y.append(label)
    return X, Y

In [3]:
X_0, Y_0 = load_dataset("data/0", 0)
X_1, Y_1 = load_dataset("data/1", 1)

In [4]:
train_rate = 0.5
train_X_0 = X_0[0:int(len(X_0)*train_rate)]
train_X_1 = X_1[0:int(len(X_1)*train_rate)]
train_Y_0 = Y_0[0:int(len(Y_0)*train_rate)]
train_Y_1 = Y_1[0:int(len(Y_1)*train_rate)]

test_X_0 = X_0[int(len(X_0)*train_rate):]
test_X_1 = X_1[int(len(X_1)*train_rate):]
test_Y_0 = Y_0[int(len(Y_0)*train_rate):]
test_Y_1 = Y_1[int(len(Y_1)*train_rate):]

train_X = numpy.r_[train_X_0, train_X_1]
train_Y = numpy.r_[train_Y_0, train_Y_1]
test_X = numpy.r_[test_X_0, test_X_1]
test_Y = numpy.r_[test_Y_0, test_Y_1]

In [5]:
train_Y[0:2]

array([0, 0])

In [6]:
train_X[0:2]

array([[[3, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 2, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=uint8)

In [7]:
model = autokeras.ImageClassifier()

In [8]:
print(len(train_X), len(train_Y))
print(train_X.shape)
print(train_Y.shape)

923 923
(923, 28, 28)
(923,)


In [9]:
model.fit(train_X, train_Y, time_limit=0.2*60*60)

In [10]:
model.cnn.searcher.history

[{'model_id': 0,
  'loss': 0.046143538504838946,
  'metric_value': 0.9868421052631579},
 {'model_id': 1,
  'loss': 0.022441903222352268,
  'metric_value': 0.9894736842105264},
 {'model_id': 2,
  'loss': 0.014146993681788445,
  'metric_value': 0.9868421052631579},
 {'model_id': 3,
  'loss': 0.07062014322727919,
  'metric_value': 0.9894736842105264}]

In [11]:
model.cnn.best_model.produce_model()

TorchModel(
  (0): Conv1d(28, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): ReLU()
  (4): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (8): ReLU()
  (9): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
  (10): TorchAdd()
  (11): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU()
  (13): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (14): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (15): ReLU()
  (16): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (17): ReLU()
  (18): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
 

In [12]:
model.final_fit(
    train_X,
    train_Y,
    train_X,
    train_Y,
    retrain=False)

In [13]:
model.predict(test_X[0:3])

array([0, 0, 0])

In [14]:
# save
path = "model.hdf5"
model.export_autokeras_model(path)

In [15]:
output_model = autokeras.utils.pickle_from_file(path)

In [16]:
vec_train_X = output_model.predict(train_X, output_index=-2)
vec_test_X = output_model.predict(test_X, output_index=-2)

In [17]:
pandas.DataFrame(vec_train_X).head(2)

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.587567,-0.149599,-0.062462,-1.012283,0.222322,0.355869,0.004161,0.653720,0.054240,-0.442793,...,-0.643198,-0.513914,0.262298,0.365572,0.973723,0.919899,0.527722,-0.784579,-0.612002,0.797502
1,0.736495,-0.061645,0.133231,-0.856721,0.276225,0.066109,0.037032,0.401691,0.047063,-0.146201,...,-0.640683,-0.389744,0.176339,0.244515,0.723510,0.661996,0.333194,-0.270008,-0.636558,0.495856


In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
lr = LogisticRegression(solver="lbfgs")

In [20]:
lr.fit(vec_test_X, test_Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [21]:
lr.score(vec_test_X, test_Y)

0.9978378378378379